<a href="https://colab.research.google.com/github/yossarian-z257/medical_image_as_graph/blob/main/ensembling_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd ~
!rm -rf medical_image_as_graph
!git clone https://github.com/yossarian-z257/medical_image_as_graph.git

/root
Cloning into 'medical_image_as_graph'...
remote: Enumerating objects: 524, done.
remote: Counting objects: 100% (370/370), done.
remote: Compressing objects: 100% (329/329), done.
remote: Total 524 (delta 68), reused 320 (delta 33), pack-reused 154
Receiving objects: 100% (524/524), 67.79 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [ ]:
%cd medical_image_as_graph
!mkdir saved_data_loader
# !rm -rf medical_image_as_graph

/root/medical_image_as_graph


In [ ]:

!pip install imageio==2.22.4
!pip install llvmlite==0.39.1
# !pip install matplotlib==3.6.2
!pip install matplotlib==3.1.3
!pip install networkx==2.8.8
!pip install numba==0.56.4
!pip install numpy
!pip install opencv-python==4.6.0.66
!pip install psutil==5.9.4
!pip install pytz==2022.6
!pip install scikit-image==0.19.3
!pip install scipy 
!pip install timm==0.6.12
!pip install torch==1.13.0
!pip install torchinfo==1.7.1
!pip install torchvision==0.14.0

import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 
# !pip install gdown
!pip install ipywidgets
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in

## IMPORTS


In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
from torch_geometric.loader import DataLoader
import os
from sklearn.model_selection import KFold
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops
from torch_geometric.nn import Sequential,GraphConv,GCNConv,TopKPooling, GatedGraphConv, JumpingKnowledge
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
import numpy as np
from torch_geometric.nn import MLP, DynamicEdgeConv, global_max_pool

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import pickle
import torch_geometric.transforms as T
# from torch_geometric.logging import init_wandb, log
from torch_geometric.nn import GCNConv
import random
from torch.nn import Linear, ReLU, Dropout
from torch_geometric.nn import global_mean_pool
import torch.nn as nn
from itertools import zip_longest
import ipywidgets as widgets
from IPython.display import display
from sklearn import  metrics 

## Experiment selection

In [ ]:
gnn_models = ['GCN', 'GAT', 'GIN']
super_pixels = ['5', '10', '50','100','150','300']
cnn_models = ['resnet18', 'efficientnet-b0', 'densenet121']

# defaults, before selection
gnn = "GAT"
sp = "10"
feature = "resnet18"


header1 = widgets.HTML(value="<b>GNN models</b>")
header2 = widgets.HTML(value="<b>Number of Superpixels</b>")
header3 = widgets.HTML(value="<b>CNN features</b>")


dropdown1 = widgets.Dropdown(options=gnn_models)
dropdown2 = widgets.Dropdown(options=super_pixels)
dropdown3 = widgets.Dropdown(options=cnn_models)

In [ ]:
def handle_dropdown1_change(change):
    global gnn
    gnn = change['new']
    
def handle_dropdown2_change(change):
    global sp
    sp = change['new']
    
def handle_dropdown3_change(change):
    global feature
    feature = change['new']
    
dropdown1.observe(handle_dropdown1_change, names='value')
dropdown2.observe(handle_dropdown2_change, names='value')
dropdown3.observe(handle_dropdown3_change, names='value')



display(widgets.VBox([header1, dropdown1]), 
        widgets.VBox([header2, dropdown2]),
        widgets.VBox([header3, dropdown3]))

In [ ]:
print(gnn,sp,feature)

GAT 10 resnet18


In [ ]:
from config import saved_data_loader
experiment_text = f"{sp}_{feature}"
for fl in ['train','test','val']:
    file_name = f"{fl}_dataloader_{experiment_text}"
    file_url =  saved_data_loader[file_name]
    file_name = file_name + ".pkl"
    os.system(f"curl -o {file_name} -L '{file_url}'")
os.system(f"mv *.pkl saved_data_loader/")

0

In [ ]:
train = "no"

!python main.py  --cnn_model_name $feature --gnn_model $gnn  --use_saved_state yes   --superpixel_number $sp  --train $train

starting main.py for saved state = True
Model name:  resnet18
loading data
Using saved state
Testing
outputs/GAT_10_resnet18_best_model.pt
number of paramteres for this model 35170
label  1 sensitivity 0.9871794871794872
Test Loss: 0.2339
Test Acc: 0.9006 %
testing complete
Done


In [ ]:
train_loader, test_loader, val_loader = None, None, None

with open(f'saved_data_loader/train_dataloader_{sp}_{feature}.pkl', 'rb') as f:
    train_loader = pickle.load(f)

with open(f'saved_data_loader/test_dataloader_{sp}_{feature}.pkl', 'rb') as f:
    test_loader = pickle.load(f)

with open(f'saved_data_loader/val_dataloader_{sp}_{feature}.pkl', 'rb') as f:
    val_loader = pickle.load(f)

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x


In [ ]:
from models import GCN,GAT,GIN


def get_gnn_model(gnn_model , input_size, ensemble = False):
    if gnn_model == "GCN":
        model = GCN(input_size)
        return model
    elif gnn_model == "GIN":
        model = GIN(input_size)
        return model
    elif gnn_model == "GAT":
        model = GAT(input_size)
        return model
    # elif ensemble == True:
    #     model = Ensemble(input_size)
    #     return model
    else:
        raise Exception("model not found")


def load_best_model(input_size, path, gcn):
    """
    load the best model
    """
    model  = get_gnn_model(gcn, input_size)
    # model = Ensemble(input_size, cnn_model, gcn)
    model.load_state_dict(torch.load(path))
    return model


class EnsembleModel(nn.Module):
    def __init__(self, modelA, modelB, modelC,num_features):
        super().__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.modelC = modelC
        self.classifier = nn.Linear(2*num_features, num_features)
        
    def forward(self, x, edge_index, batch):
        x1 = self.modelA(x, edge_index,batch)
        x2 = self.modelB(x, edge_index, batch)
        x3 = self.modelC(x, edge_index,batch)
        x3x = x3#torch.cat((x3[0],x3[1]),dim = 1)
        x = torch.cat((x1, x2, x3x), dim=1)
        x = self.classifier(F.relu(x))
        return x

def Ensemble(cnn_model):
    """
    Ensembly of top 3 models, top 3 of differnt architecture.
    """

    if cnn_model == 'denset121':
        modelb = load_best_model(1024,f'outputs/GAT_10_densenet121_best_model.pt','GAT')
        modela = load_best_model(1024,f'outputs/GCN_10_densenet121_best_model.pt','GCN')
        modelc = load_best_model(1024,f'outputs/GIN_150_densenet121_best_model.pt','GIN')

    elif cnn_model == 'efficientnet-b0':
        modelb = load_best_model(1280,f'outputs/GAT_10_efficientnet-b0_best_model.pt','GAT')
        modela = load_best_model(1280,f'outputs/GCN_10_efficientnet-b0_best_model.pt','GCN')
        modelc = load_best_model(1280,f'outputs/GIN_10_efficientnet-b0_best_model.pt','GIN')

    else:
        modelb = load_best_model(512,f'outputs/GAT_50_resnet18_best_model.pt','GAT')
        modela = load_best_model(512,f'outputs/GCN_10_resnet18_best_model.pt','GCN')
        modelc = load_best_model(512,f'outputs/GIN_10_resnet18_best_model.pt','GIN')

    # rename after experiments, docs todo
    modela.lin = Identity()
    modelb.lin1 = Identity()
    # model3.lin1 = Identity()
    modelc.lin2 = Identity()

    model = EnsembleModel(modela, modelb, modelc, 2)
    model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.5, inplace=False), 
    torch.nn.Linear(in_features=128, 
                    out_features=2,
                    bias=True))
                    #.to(device)

    return model

model = Ensemble(feature)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.001)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def train():
    model.train()

    for data in train_loader: 
        
        out = model(data.x, data.edge_index, data.batch)  
        data.y = torch.Tensor(data.y)
        data.y = torch.Tensor(torch.flatten(data.y))
        data.y = data.y.type(torch.LongTensor)

        loss = criterion(out, data.y)
        loss.backward()  
        optimizer.step() 
        optimizer.zero_grad()  
cfm = None
label = []
predication = []
def test(loader, flag = 0):
     model.eval()

     correct = 0
     for data in loader:  
         out = model(data.x, data.edge_index, data.batch)  
         data.y = torch.Tensor(data.y)

         pred = out.argmax(dim=1).view(-1,1)  
         cf_matrix = confusion_matrix(data.y,pred)
         global cfm
         cfm = cf_matrix
         if flag:
            print(cfm)
         correct += int((pred == data.y).sum()) 
         acc = correct / len(loader.dataset) 
         specificity_score = metrics.recall_score(data.y, pred, pos_label=0)
         sensitivity_score2 = metrics.recall_score(data.y, pred, pos_label=1)
         if flag:
            print("specificity_score",specificity_score,"sensitivity_score",sensitivity_score2)
         if flag:
            print(f"ROCAUC: {roc_auc_score(data.y.cpu().numpy(),pred.cpu().numpy(),average=None)}")
     return correct / len(loader.dataset) 


for epoch in range(1, 21):
    train()
    try:
        train_acc = test(train_loader)
        test_acc = test(test_loader,1)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        
    except Exception as e:
        print("error",e)
        pass
print("check out sensitivity score for ensempled models")
print("number of paramteres for this model",sum(p.numel() for p in model.parameters()))

[[117 117]
 [  0 390]]
specificity_score 0.5 sensitivity_score 1.0
ROCAUC: 0.75
Epoch: 001, Train Acc: 0.9630, Test Acc: 0.8125
[[127 107]
 [  1 389]]
specificity_score 0.5427350427350427 sensitivity_score 0.9974358974358974
ROCAUC: 0.7700854700854701
Epoch: 002, Train Acc: 0.9668, Test Acc: 0.8269
[[146  88]
 [  2 388]]
specificity_score 0.6239316239316239 sensitivity_score 0.9948717948717949
ROCAUC: 0.8094017094017094
Epoch: 003, Train Acc: 0.9676, Test Acc: 0.8558
[[143  91]
 [  1 389]]
specificity_score 0.6111111111111112 sensitivity_score 0.9974358974358974
ROCAUC: 0.8042735042735043
Epoch: 004, Train Acc: 0.9689, Test Acc: 0.8526
[[132 102]
 [  0 390]]
specificity_score 0.5641025641025641 sensitivity_score 1.0
ROCAUC: 0.782051282051282
Epoch: 005, Train Acc: 0.9697, Test Acc: 0.8365
